In [1]:
import os
import gc
import cv2
import copy
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L

# Utils
from tqdm import tqdm
from IPython.display import display

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [2]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        
#         in_features = self.model.classifier.in_features
#         self.model.classifier = nn.Identity()
        
        in_features = self.model.head.in_features
        self.model.head = nn.Identity()
        
        self.fc = nn.Linear(in_features, cls)
        
    def forward(self, image):
        x = self.model(image)
        x = self.fc(x)
        return x

In [3]:
def get_train_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Blur(blur_limit= 3, p=0.3),
        A.GaussNoise(p=0.3),
#         A.OneOf([
#             A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
#             A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
#         ], p=0.3),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit= 30,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        
        ToTensorV2(p=1.0),
    ])

def get_2s_train_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

In [4]:
class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.image_path = df['image_path'].values
        self.labels = df['class_num'].values
        self.transforms = transforms
    
    def __getitem__(self, index):
        path = self.image_path[index]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = int(self.labels[index])
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img/255, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.long),
        }
    
    def __len__(self):
        return len(self.df)

In [5]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.CrossEntropy= nn.CrossEntropyLoss()
        self.SoftCrossEntropy= L.SoftCrossEntropyLoss(smooth_factor= 0.2)
    def forward(self, y_pred, y_true):
        loss= self.SoftCrossEntropy(y_pred, y_true)
        return loss

In [6]:
def train_epoch(dataloader, model, criterion, optimizer):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        
        with amp.autocast():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
            loss/= CFG['gradient_accumulation']
            scaler.scale(loss).backward()
            
            if (i+1) % CFG['gradient_accumulation']== 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
    return np.mean(ep_loss)

In [7]:
def WP(all_pred, all_label):
    df= pd.DataFrame(columns= ('class', 'recall', 'precision', 'f1_score'))
    WP= 0
    for cls in range(1, 16):
        recall= all_pred[all_label==cls].tolist().count(cls) / all_label.tolist().count(cls)
        precision= all_label[all_pred==cls].tolist().count(cls) / all_pred.tolist().count(cls)
        f1_score= (2 * precision * recall) / (precision + recall)
        WP+= precision*all_label.tolist().count(cls)
        df.loc[cls-1]= [int(cls), round(recall,3), round(precision,3), round(f1_score,3)]
    WP/= len(all_label)
    display(df)
    print(f'WP= {WP}')
    return WP


def valid_epoch(dataloader, model, criterion):
    model.eval()
    
    ep_loss= []
    all_pred= []
    all_label= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        all_label.extend(labels.cpu().numpy())
        
        with torch.no_grad():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
        all_pred.extend(preds.cpu().softmax(dim=-1).numpy().argmax(1))
        
    ## metrice
    wp= WP(np.array(all_pred), np.array(all_label))
    
    return np.mean(ep_loss), wp

# CFG

In [8]:
CFG= {
    'fold': 0,
    'epoch': 10,
    'model_name': 'swin_base_patch4_window12_384',
    'finetune': False,
    
    'img_size': 384,
    'batch_size': 8,
    'gradient_accumulation': 1,
    
    'lr': 5e-5,
    'weight_decay': 0,
    
    'num_classes': 16,
    'load_model': False,
    'save_model': './Model/train'
}

if CFG['finetune']:
    print('finetune model')
    CFG['load_model']= f"Model/train/cv{CFG['fold']}_best.pth"
    CFG['epoch']= 5
    CFG['lr']= 5e-6

# Prepare Dataset

In [9]:
df= pd.read_csv('Dataset/train.csv')
train_df= df[df['fold']!=CFG['fold']]
valid_df= df[df['fold']==CFG['fold']]
print(f'train dataset: {len(train_df)}')
print(f'valid dataset: {len(valid_df)}')

train_df= pd.concat([train_df, valid_df],axis=0)
if CFG['finetune']:
    train_dataset= Customize_Dataset(train_df, get_2s_train_transform(CFG['img_size']))
else:
    train_dataset= Customize_Dataset(train_df, get_train_transform(CFG['img_size']))
valid_dataset= Customize_Dataset(valid_df, get_test_transform(CFG['img_size']))

train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
valid_loader= DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=0)
df.head()

train dataset: 64216
valid dataset: 16054


,image_path,class,class_num,fold
0,Dataset/train_img\banana-013\banana\160107-3-0...,banana,13,2
1,Dataset/train_img\banana-013\banana\160107-3-0...,banana,13,2
2,Dataset/train_img\banana-013\banana\160107-3-0...,banana,13,0
3,Dataset/train_img\banana-013\banana\160107-3-0...,banana,13,0
4,Dataset/train_img\banana-013\banana\160107-3-0...,banana,13,3


# Train

In [ ]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= Customize_Model(CFG['model_name'], CFG['num_classes'])
model.to('cuda')
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
best_score= 0
for ep in range(1, CFG['epoch']+1):
    print(f'ep: {ep}')
    
    train_loss= train_epoch(train_loader, model, criterion, optimizer)
    valid_loss, valid_acc= valid_epoch(valid_loader, model, criterion)
    print(f'train loss: {train_loss}')
    print(f'valid loss: {valid_loss}')
    
    if valid_acc >= best_score:
        best_score= valid_acc
        torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_swin_best.pth")
        print(f'model save at score: {best_score}')
        
    ## save model every epoch
    torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")
    
    ## adjust lr
    if ep == 7:
        model= torch.load(f"{CFG['save_model']}/cv{CFG['fold']}_best.pth")
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease learning rate to 1e-4!')